In [1]:
from TripletLossFunc import TripletLossFunc
import random

import torch
import torch.nn.functional as F

import torchvision.models as models

from Utils import *
from Define import *
from NUS_WIDE_Helper import *

from jupyterplot import ProgressPlot
from tqdm.notebook import tqdm
from TenNetImage import *
from TenNetTag import *
from TagDecoder import *
import TenNet

In [2]:
#NUS_WIDE
train_data = NUS_WIDE_Helper(DataSetType.Train_81,)
valid_data = NUS_WIDE_Helper(DataSetType.Test_81,  Number_Of_Images_Valid)
#test_data = NUS_WIDE_Helper(DataSetType.Test_81)

batch_size = BATCH_SIZE
train_loader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = torch.utils.data.DataLoader(valid_data, shuffle=True, batch_size=batch_size)
#test_loader = torch.utils.data.DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [3]:
#image_model = models.vgg16(pretrained=True).to(device)
image_model = TenNet_Image().to(device)
tag_model = TenNet_Tag(train_data.get_tag_list()).to(device)

optim = torch.optim.Adam([{'params' : image_model.parameters()}, {'params' : tag_model.parameters()}], lr=0.001)
triplet_loss = TripletLossFunc(Margin_Distance)

In [4]:
# debug list
max_loss = Margin_Distance * 2
n_epochs = N_Epochs
Lambda = 0.3
min_valid_loss = -1

In [2]:
pp = ProgressPlot(plot_names=["loss", "train distance"],
                  line_names=["train/pos_IT", "valid/pos_II", "0/neg_IT", "0/neg_II"],
                  x_lim=[0, n_epochs-1], 
                  y_lim=[0, max_loss])

pbar = tqdm(range(n_epochs))

for e in pbar:
    
    loss_dis_train =  TenNet.train(image_model, tag_model, train_loader, triplet_loss, Lambda, optim)
    TenNet.output_loss_dis(f"epoch:{e}: 1-train dataset with train model", loss_dis_train)
        
    loss_dis_valid = TenNet.evalue(image_model, tag_model, valid_loader, triplet_loss, Lambda, optim, e, min_valid_loss, True)   
    TenNet.output_loss_dis(f"epoch:{e}: 2-valid dataset with evalue model", loss_dis_valid)
    min_valid_loss = loss_dis_valid[5]

    pp.update([ [min(loss_dis_train, max_loss), min(loss_dis_valid, max_loss), 0, 0],
                [min(loss_dis_train[1], max_dis), 
                 min(loss_dis_train[2], max_dis), 
                 min(loss_dis_train[3], max_dis), 
                 min(loss_dis_train[4], max_dis)]])
pp.finalize()


NameError: name 'n_epochs' is not defined